In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of BERT_tasks failed: Traceback (most recent call last):
  File "/dfs/scratch0/vschen/venv-mmtl/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "/dfs/scratch0/vschen/venv-mmtl/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 376, in superreload
    module = reload(module)
  File "/dfs/scratch0/vschen/venv-mmtl/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/dfs/scratch0/vschen/venv-mmtl/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/dfs/scratch0/vschen/metal-mmtl/metal/mmtl/BERT_tasks.py", line 6, in <module>
    from dataset import COLADataset, SST2Dataset, STSBDataset

In [6]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [7]:
import torch.nn as nn
from metal.mmtl.task import Task
from BERT_tasks import create_task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000)
bar_data = make_dataloader(2000)

foo = Task("foo_task", {"train":foo_data, "valid":foo_data, "test":None}, foo_input, foo_head, )
bar = create_task('SST-2')
baz = create_task('CoLA')
tasks = [foo,baz]

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

tasks = [bar]
model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=3, 
    lr=0.005, 
    progress_bar=True,
    log_every=1,
    score_every=2,
    checkpoint_best=True,
    checkpoint_metric="SST-2/valid/accuracy",
    checkpoint_metric_mode="max",
)